## Import Statements

In [228]:
import gensim.downloader as api
import numpy as np
import re
import csv
import pandas as pd
import pprint
import string
import nltk
import sys
import spacy
    
from IPython.display import HTML
from nltk.corpus import wordnet 
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize

pathToDatasets = '../datasets/'
pathToDataScripts = '../datasets/scripts/'
filePath = '../datasets/GoogleNews-vectors-negative300.bin'

sys.path.insert(0, pathToDataScripts)
from cleanDataset import tokenize_words 


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

## Downloading binaries and models


In [229]:
print("Should I reload the model?")
tstString = input()
if("no" in tstString.lower() or "n" in tstString.lower()):
    print(" didnt reload model! ")
else:
    print("loading the model!");
    word_vectors = api.load("glove-wiki-gigaword-300")
    nltk.download('vader_lexicon')
    nltk.download('punkt')
    nltk.download('averaged_perceptron_tagger')
    nltk.download('wordnet')
    print("Model Loaded!")

Should I reload the model?
yeet
loading the model!
Model Loaded!


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Global Variables and Global Objects

In [230]:
senty = SentimentIntensityAnalyzer()
vocabulary = word_vectors.vocab;

nlp = spacy.load('en')


NUMBER_OF_ALTERNATIVES = 5
TWEET_START = 1
NUM_OF_TWEETS = 10


VERBOSE_PRINTING = True
# VERBOSE_PRINTING = False

# USE_SPACY = False
USE_SPACY = True

COLOR_PRINTING = True
#COLOR_PRINTING = False

# PRINT_NEUTRAL = True
PRINT_NEUTRAL = False

PRINT_ALL_STRINGS = True
# PRINT_ALL_STRINGS = False

SHOW_ALTS = 30

punctuation = r"\"#$%&'+-/;<=>?@[\]*^_`{|}~"

## Class for Sentences


In [231]:
from SentenceClass import *

Imported class file!


## Utility Code

In [232]:
def printStrings(sentenceObj):
    
    numberOfPrints = 0
    newStrings = generateHTMLObjectsFromSentence(sentenceObj)
    mainSentiment = sentenceObj.ogSentiment;
    listOfSentencesWithSentiments = []
    bestSentiment = -sys.maxsize - 1
    worstSentiment = sys.maxsize
    
    bestSentimentString = "";
    worstSentimentString = "";
    
    
    
    for ind, tSentence in enumerate(newStrings):
            alteredTweet = tSentence.getSentence()
            htmlText = tSentence.getHTML()
            sentimentOfNewString = senty.polarity_scores(alteredTweet)['compound']
            listOfSentencesWithSentiments.append(SentenceWithSentiment(alteredTweet, sentimentOfNewString, htmlText))
            
            if(sentimentOfNewString >= bestSentiment):
                bestSentiment = sentimentOfNewString
                bestSentimentString = htmlText;
            elif(sentimentOfNewString < worstSentiment):
                worstSentiment = sentimentOfNewString
                worstSentimentString = htmlText;
                
            
            if(sentimentOfNewString == mainSentiment or sentimentOfNewString == 0.0):
                if(PRINT_ALL_STRINGS and numberOfPrints <= SHOW_ALTS): displayText("{0}: {1}".format(htmlText,sentimentOfNewString),'black')
                numberOfPrints+=1;
            elif(sentimentOfNewString > mainSentiment):
                if(PRINT_ALL_STRINGS and numberOfPrints <= SHOW_ALTS): displayText("{0}: {1}".format(htmlText,sentimentOfNewString),'green')
                numberOfPrints+=1;
            elif(sentimentOfNewString < mainSentiment and sentimentOfNewString != 0.0):
                if(PRINT_ALL_STRINGS and numberOfPrints <= SHOW_ALTS): displayText("{0}: {1}".format(htmlText,sentimentOfNewString),'red')
                numberOfPrints+=1;
                
    print("\n")
    if(numberOfPrints > SHOW_ALTS): print("--- More options (total: {0}) possible, but not printed ---".format(numberOfPrints));
    if (worstSentimentString != ""): displayText("Worst Sentence: {0} : {1}".format(worstSentimentString, worstSentiment), color='red')
    if (bestSentimentString != ""): displayText("Best Sentence: {0} : {1}".format(bestSentimentString, bestSentiment), color='green') 
    print("\n")
    
    
    return listOfSentencesWithSentiments;

def cstr(s, color='black', italics=False):
    if(COLOR_PRINTING):
        if(italics):
            return cstr("<i>{0}</i>".format(s), color);
        return "<text style=color:{}>{}</text>".format(color, s)
    else:
        return "{}".format(s)

def displayText(text, color='black'):
    if(COLOR_PRINTING):
        display(HTML(cstr(text, color)));
        return
    print("{}".format(text));
    
    
def cleanAndTokenizeText(text):
    text = text.lower();
    newString = ""
    for char in text:
        if char not in punctuation:
            newString += char
    text = word_tokenize(newString)
    return text;

def getPOSTags(tweet):
    if(USE_SPACY == False):
        tags = nltk.pos_tag(tweet)
        return tags;    
    tweet = ' '.join(tweet)
    doc = nlp(tweet)
    tags = [(token.text, token.tag_) for token in doc] # since the format expected is [text,tag]
    return tags

def getAntonymsOfWords(word):
    if(word not in vocabulary):
        return []
    setOfAntonyms = set()
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            anton = l.antonyms()
            if(anton!=[]):
                setOfAntonyms.add(anton[0].name())
    if(len(setOfAntonyms) == 0):
        if(VERBOSE_PRINTING): print("No antonyms found for word {0}".format(word))
    return list(setOfAntonyms)

def returnReplacementsForWord(word):
    
    if(word not in vocabulary):
        print(" --- {0} not in vocabulary ---".format(word))
        return []
    possibleReplacements = [word[0] for word in word_vectors.most_similar(word,topn=NUMBER_OF_ALTERNATIVES)]
    
    if(possibleReplacements == []):
        print(" --- No replacements for word {0} ---".format(word))
    antonyms = getAntonymsOfWords(word)
    if(antonyms != []):
        possibleReplacements.extend(antonyms)
        if(VERBOSE_PRINTING): print("Some antonyms for word {0} are {1}".format(word, antonyms[:3]))
        return possibleReplacements
    return possibleReplacements
    
def posApprovedReplacements(alternativeWords, userTokens, indexOfToken):
    if(alternativeWords == []):
        return []
    tempTokens = userTokens[:]
    POSTokens = getPOSTags(tempTokens)
    validWords = []
    
    mainTag = POSTokens[indexOfToken][1]
    mainWord = userTokens[indexOfToken]
    
    for ind,word in enumerate(alternativeWords):
        tempTokens[indexOfToken] = word
        posTags = getPOSTags(tempTokens)
        newTag = (posTags[indexOfToken])[1]
        
        if(str(newTag) == str(mainTag)):
            if(VERBOSE_PRINTING): print("Word {0}[{1}] replaced with {2}[{3}]".format(mainWord, mainTag, word,newTag))
            validWords.append(word)
    if(validWords == [] and VERBOSE_PRINTING):
        print("No POS words found for word {} with tag {}".format(mainWord, mainTag));
    return validWords
    
def generateHTMLObjectsFromSentence(sentenceObj):
    
    allSentences = sentenceObj.getFinalSentences()
    indexToAlts = sentenceObj.indexToSetOfWords;
    indexToChange = list(indexToAlts.keys());
    
    listOfSentenceObjs = []
    for sentence in allSentences:
        copySentence = cleanAndTokenizeText(sentence)
        for index in indexToChange:
            copySentence[index] = cstr("[{0}]".format(copySentence[index]), "blue", italics=True);
        listOfSentenceObjs.append(SentenceWithHTML(sentence, ' '.join(copySentence)));
    
    return listOfSentenceObjs
    
    
    

## Sentence Chunking and Appending

In [233]:
def helper_combine(mainList, myList):
    '''
    helper function for CombineSentenceChunks
    '''
    newList = []
    for val in myList:
        for mainVal in mainList:
#             if(VERBOSE_PRINTING): print("Combining {0} with {1}".format(' '.join(val), ' '.join(mainVal)));
            newList.append(val + mainVal);
    return newList;

def combineSentenceChunks(wholeSentence, dictOfChunks):
    '''
        Uses the helper_combine function to generate all possible combinatios and permuations of the chunks
        and any alternatives.
        
        The logic is to use the end of the sentence, apply each possible chunk from the previous key's chunks
        to every possible chunk of this key's.
        
        The helper function is used to allow us to reuse the list of alreadyGeneratedChunks and constantly
        append to them.
        
        To understand the logic better, take a look at this gist:
        https://gist.github.com/sunnyMiglani/cf85407a9e6928237b1436cc2bc95fa4
        
    '''
    reversedKeys = sorted(dictOfChunks.keys(), reverse=True)
    completeSentences = [];
    mainArr = dictOfChunks[reversedKeys[0]]
    for ind in range(1, len(reversedKeys)):
        key = reversedKeys[ind];
        mainArr = helper_combine(mainArr, dictOfChunks[key]);
        
    return mainArr;
        
def generateSentenceChunks(wholeSentence, keyToChange, nextKey, listOfMyAlternatives):
    '''
        Generates sentence chunks by iterating through the list of alternatives
        Chunking the sentence to start from current key to next key.
        This means that the sentence always goes from key 'x' to key 'y'
        
        Example:
        "I really <hate> hot chocolate, but I <prefer> hot coffee"
        Calling generateSentenceChunks will create an example sentence:
            - "<altWordForHate> hot chocolate , but I "
        
        Remember to append the first stretch of the string to the first key's chunk for proper use!
    '''
    newList = list(listOfMyAlternatives)
    newList.append(wholeSentence[keyToChange]);
    generatedSentences = []
    for myAlt in newList:
        newSentence = wholeSentence[:]
        newSentence[keyToChange] = myAlt
        if(VERBOSE_PRINTING): print("Generated : {}".format(newSentence[keyToChange:nextKey]))
        generatedSentences.append(newSentence[keyToChange:nextKey]);
        
    return generatedSentences
    
def returnCombinationsOfStrings(sentenceObj):
    
    indexToWordDict = sentenceObj.indexToSetOfWords;
    originalSentence = sentenceObj.ogSentence;
    tokenizedSentence = cleanAndTokenizeText(originalSentence)
    reversedKeys = sorted(indexToWordDict.keys(), reverse=True)
    dictAlternatives  = {}

    sortedKeys = sorted(indexToWordDict.keys())
    sentenceChunks = {}
    htmlChunks = {}
    
    for ind in range(0,len(sortedKeys)):
        key = sortedKeys[ind]
        nextKey = sortedKeys[ind+1] if ind+1 < len(sortedKeys) else len(tokenizedSentence)
        sentenceChunks[key] = generateSentenceChunks(tokenizedSentence, key, nextKey, indexToWordDict[key])

    if(sortedKeys[0] != 0):
        newList = []
        for thislist in sentenceChunks[sortedKeys[0]]:
            newList.append(tokenizedSentence[0:sortedKeys[0]] + thislist)
        sentenceChunks[sortedKeys[0]] = newList;
        
    finalOptions = combineSentenceChunks(tokenizedSentence, sentenceChunks)
    
    finalSentences = []
    for val in finalOptions:
        sentence = ' '.join(val)
        finalSentences.append(sentence)
    
    sentenceObj.resetFinalSentences()
    sentenceObj.addFinalSentences(finalSentences)   
    return sentenceObj

In [234]:
def getAlternativeSentences(sentenceObj):
    mainSentence = sentenceObj.ogSentence;
    mainSentiment = sentenceObj.ogSentiment;

    sentenceTokens = cleanAndTokenizeText(mainSentence)

    for ind, word in enumerate(sentenceTokens):
        alternativeSentenceWithHTML = []
        copyOfTokens = sentenceTokens[:]
        replacements = []
        
        score = senty.polarity_scores(word)['compound']
        if(score != 0.0):
            tempReplacements = returnReplacementsForWord(word) # get embedding based relations
            if(tempReplacements == []):
                print("No replacements found at all for word {0}".format(word))
                continue
            replacements = posApprovedReplacements(tempReplacements[:], copyOfTokens[:], ind)
            finalReplacements = []
            if(VERBOSE_PRINTING): skippedWords = []
            for word in replacements:
                thisSentiment = senty.polarity_scores(word)['compound']
                if(thisSentiment != 0.0):
                    finalReplacements.append(word)
                else:
                    if(VERBOSE_PRINTING):
#                         print("Sentiment of Skipped word {} is {}".format(word, senty.polarity_scores(word)))
                        skippedWords.append(word)
            if(VERBOSE_PRINTING and len(skippedWords) > 0):
                print("some skipped words: {0}".format(skippedWords[:5]));
            if(finalReplacements == []):
                print(" -- No POS approved words! -- for word {0}\n some non-POS:{1}".format(word, tempReplacements[:4]))
                continue
            sentenceObj.addAlternativesByIndex(ind, finalReplacements)
    return sentenceObj




In [235]:
def extractTwitterDataset():
    df_tweets = pd.read_csv( pathToDatasets + 'cleanedTweets.csv', nrows=NUM_OF_TWEETS, skiprows =TWEET_START)
    tweets = df_tweets.values
    return tweets;


def runThroughTweets():
    tweets = extractTwitterDataset()
    
    counterOfTweets = 0;
    for counter,tweet in enumerate(tweets):
        counterOfTweets +=1;
        tweet = tweet[0]
        tweetTokens = cleanAndTokenizeText(tweet)
        mainSentiment = senty.polarity_scores(tweet)['compound']
        if(mainSentiment == 0):
            continue
        print("\n {0}:{1}\n".format(tweet,mainSentiment))   
        sentenceObj = Sentence(tweet, mainSentiment)
        sentenceObj = getAlternativeSentences(sentenceObj)
        replacementDictionary = sentenceObj.getDictOfIndexWords();
        if(len(replacementDictionary) <= 0):        
            print(" -- No new Strings generated ---\n\n")
            continue

        keysToChange = replacementDictionary.keys();
        for key in keysToChange:
            if(VERBOSE_PRINTING):print("{0}'th word's ({2}) options: {1}".format(key,replacementDictionary[key],tweetTokens[key]))

        sentenceObj = returnCombinationsOfStrings(sentenceObj)
        allPossible = printStrings(sentenceObj)
    return counterOfTweets;
        
numOfTweets = runThroughTweets()
print("num of tweets done : {0}".format(numOfTweets))


 is upset that he can not update his facebook by texting it... and might cry as a result  school today also. blah!:-0.75

No antonyms found for word upset
Word upset[VBN] replaced with shocked[VBN]
Some antonyms for word cry are ['laugh']
Word cry[VB] replaced with scream[VB]
Word cry[VB] replaced with shout[VB]
Word cry[VB] replaced with laugh[VB]
some skipped words: ['shout']
No antonyms found for word blah
Word blah[UH] replaced with goy[UH]
Word blah[UH] replaced with vich[UH]
some skipped words: ['goy', 'vich']
 -- No POS approved words! -- for word vich
 some non-POS:['goy', 'stuh', 'blagojevich', 'vich']
1'th word's (upset) options: {'shocked'}
15'th word's (cry) options: {'scream', 'laugh'}
Generated : ['shocked', 'that', 'he', 'can', 'not', 'update', 'his', 'facebook', 'by', 'texting', 'it', '...', 'and', 'might']
Generated : ['upset', 'that', 'he', 'can', 'not', 'update', 'his', 'facebook', 'by', 'texting', 'it', '...', 'and', 'might']
Generated : ['scream', 'as', 'a', 'resu




  i dived many times for the ball. managed to save 50  the rest go out of bounds:0.4939

No antonyms found for word save
Word save[VB] replaced with help[VB]
Word save[VB] replaced with effort[VB]
some skipped words: ['effort']
10'th word's (save) options: {'help'}
Generated : ['help', '50', 'the', 'rest', 'go', 'out', 'of', 'bounds']
Generated : ['save', '50', 'the', 'rest', 'go', 'out', 'of', 'bounds']





 my whole body feels itchy and like its on fire :-0.25

No antonyms found for word itchy
Word itchy[JJ] replaced with scratchy[JJ]
Word itchy[JJ] replaced with runny[JJ]
some skipped words: ['scratchy', 'runny']
 -- No POS approved words! -- for word runny
 some non-POS:['scratchy', 'itching', 'blisters', 'rashes']
Some antonyms for word like are ['unlike', 'dislike', 'unalike']
Word like[IN] replaced with unlike[IN]
some skipped words: ['unlike']
 -- No POS approved words! -- for word unlike
 some non-POS:['such', 'even', 'you', '?']
Some antonyms for word fire are ['hire']
Word fire[NN] replaced with blaze[NN]
Word fire[NN] replaced with firing[NN]
Word fire[NN] replaced with hire[NN]
some skipped words: ['blaze', 'hire']
9'th word's (fire) options: {'firing'}
Generated : ['firing']
Generated : ['fire']





  no it is not behaving at all. i am mad. why am i here because i can not see you all over there. :-0.6597

Some antonyms for word no are ['yes', 'all']
Word no[UH] replaced with yes[UH]
No antonyms found for word mad
Word mad[JJ] replaced with bovine[JJ]
Word mad[JJ] replaced with bse[JJ]
Word mad[JJ] replaced with spongiform[JJ]
Word mad[JJ] replaced with crazy[JJ]
some skipped words: ['bovine', 'bse', 'spongiform']
0'th word's (no) options: {'yes'}
10'th word's (mad) options: {'crazy'}
Generated : ['yes', 'it', 'is', 'not', 'behaving', 'at', 'all', '.', 'i', 'am']
Generated : ['no', 'it', 'is', 'not', 'behaving', 'at', 'all', '.', 'i', 'am']
Generated : ['crazy', '.', 'why', 'am', 'i', 'here', 'because', 'i', 'can', 'not', 'see', 'you', 'all', 'over', 'there', '.']
Generated : ['mad', '.', 'why', 'am', 'i', 'here', 'because', 'i', 'can', 'not', 'see', 'you', 'all', 'over', 'there', '.']





 need a hug :0.4767

No antonyms found for word hug
Word hug[NN] replaced with goodbye[NN]
Word hug[NN] replaced with kiss[NN]
some skipped words: ['goodbye']
2'th word's (hug) options: {'kiss'}
Generated : ['kiss']
Generated : ['hug']





  hey  long time no see! yes.. rains a bit only a bit  lol  i am fine thanks  how is you :0.68

Some antonyms for word no are ['yes', 'all']
Word no[UH] replaced with yes[UH]
No antonyms found for word lol
Word lol[NN] replaced with choua[NN]
Word lol[NN] replaced with coxhill[NN]
Word lol[NN] replaced with creme[NN]
Word lol[NN] replaced with 10cc[NN]
Word lol[NN] replaced with godley[NN]
some skipped words: ['choua', 'coxhill', 'creme', '10cc', 'godley']
 -- No POS approved words! -- for word godley
 some non-POS:['choua', 'coxhill', 'creme', '10cc']
Some antonyms for word fine are ['coarse']
Word fine[VBP] replaced with fines[VBP]
Word fine[VBP] replaced with art[VBP]
Word fine[VBP] replaced with well[VBP]
Word fine[VBP] replaced with arts[VBP]
Word fine[VBP] replaced with 'll[VBP]
Word fine[VBP] replaced with coarse[VBP]
some skipped words: ['fines', 'art', 'arts', "'ll", 'coarse']
No antonyms found for word thanks
Word thanks[JJ] replaced with despite[JJ]
Word thanks[JJ] replac



num of tweets done : 10


## Testing Code


In [236]:
specificWord = "good"
def testOneWord(word=""):
    if(word==""):
        return
    print(word)

In [237]:
specificString = ""
def specificString(textString=""):
    if(textString == "" or textString == None):
        return
    mainSentiment = senty.polarity_scores(textString)['compound']
    if(mainSentiment == 0):
        print("{} \n No sentiment found in sentence".format(textString));
        return;
    print("\n {0}:{1}\n".format(textString,mainSentiment))   
    sentenceObj = Sentence(textString, mainSentiment)
    sentenceObj = getAlternativeSentences(sentenceObj)
    replacementDictionary = sentenceObj.getDictOfIndexWords();
    if(len(replacementDictionary) <= 0):        
        print(" -- No new Strings generated ---\n\n")
        return
    
    keysToChange = replacementDictionary.keys();
    for key in keysToChange:
        print("{0}'th word's options: {1}".format(key,replacementDictionary[key]))
        
    sentenceObj = returnCombinationsOfStrings(sentenceObj)
    allPossibleSentences = printStrings(sentenceObj)
    

specificString("I am the BEST builder, just look at what I've built")
specificString("I am least racist person there is")


 I am the BEST builder, just look at what I've built:0.7125

Some antonyms for word best are ['badly', 'ill', 'evil']
Word best[JJS] replaced with worst[JJS]
3'th word's options: {'worst'}
Generated : ['worst', 'builder', ',', 'just', 'look', 'at', 'what', 'ive', 'built']
Generated : ['best', 'builder', ',', 'just', 'look', 'at', 'what', 'ive', 'built']





 I am least racist person there is:0.4973

No antonyms found for word racist
Word racist[JJ] replaced with homophobic[JJ]
Word racist[JJ] replaced with sexist[JJ]
Word racist[JJ] replaced with semitic[JJ]
Word racist[JJ] replaced with xenophobic[JJ]
some skipped words: ['homophobic', 'sexist', 'semitic', 'xenophobic']
 -- No POS approved words! -- for word xenophobic
 some non-POS:['homophobic', 'sexist', 'semitic', 'xenophobic']
 -- No new Strings generated ---


